# Comprobación de Confs de una ruta vs el WIP

El resultado del analisis se guarda en `./comprobaciones/`, la información extraida de los confs hammurabi se guarda en `./conf_info.csv`

1) necesario actualizar `wip_final.csv` para que el analisis sea actualizado
2) Indicar la ruta donde se almacenan los confs en `ruta_confs` y q a analizar en `q`


In [1]:
# !pip install --trusted-host=pypi.org --trusted-host=files.pythonhosted.org --user pyhocon

In [2]:
from os import walk
from os.path import join
from pyhocon.config_parser import STR_SUBSTITUTION
from pyhocon import ConfigFactory
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)

def get_files_path_in_directory(directory_path):
    return [join(root, file).replace('\\', '/')  for root, dirs, files in walk(directory_path) for file in files if str(file).endswith('.conf')]

def get_repo_info(directory_path):
    archivos=[]
    namings = []
    types_rule =[]
    tablas = []
    ids = []
    criticals = []
    datavaluescolumn = []
    datavaluessubset = []
    datavaluespath = []
    values = []
    subset = []
    drilldown = []
    acceptanceMin = []

    confs=get_files_path_in_directory(directory_path)
    for conf in confs:
        conf_data: ConfigFactory = ConfigFactory.parse_file(conf, resolve=False, unresolved_value=STR_SUBSTITUTION)
        tabla = conf_data["hammurabi.dataFrameInfo.physicalTargetName"]
        for rule in conf_data["hammurabi.rules"]:
            print(rule)
            archivos.append(conf)
            tablas.append(tabla)
            #id
            ids.append(rule["config.id"])
            #column(s)
            if 'column' in rule["config"].keys():
                namings.append(rule["config.column"])
            elif 'columns' in rule["config"].keys():
                namings.append(rule["config.columns"])
            else:
                namings.append(np.nan)
            #isCritical
            criticals.append(rule["config.isCritical"])
            #class
            if rule["class"]!="com.datio.hammurabi.rules.TemporalRule":
                types_rule.append(rule["class"])
            elif rule["class"] == "com.datio.hammurabi.rules.TemporalRule":
                types_rule.append(rule["config.parentClass"])
            else:
                types_rule.append(np.nan)
            #dataValuesColumn (3.5)
            if 'dataValuesColumn' in rule["config"].keys():
                datavaluescolumn.append(rule["config.dataValuesColumn"])
            else:
                datavaluescolumn.append(np.nan)
            #dataValuesSubset (3.5)
            if 'dataValuesSubset' in rule["config"].keys():
                datavaluessubset.append(rule["config.dataValuesSubset"])
            else:
                datavaluessubset.append(np.nan)
            # dataValues.Path (3.5)
            if 'dataValues' in rule["config"].keys():
                if 'paths' in rule["config"]["dataValues"].keys():
                    datavaluespath.append(rule["config.dataValues.paths"])
                elif 'tables' in rule["config"]["dataValues"].keys():
                    datavaluespath.append(rule["config.dataValues.tables"])
                else:
                    datavaluespath.append(np.nan)
            else:
                datavaluespath.append(np.nan)
            # values (3.5 static)
            if 'values' in rule["config"].keys():
                values.append(rule["config.values"])
            else:
                values.append(np.nan)
            # subset
            if 'subset' in rule["config"].keys():
                subset.append(rule["config.subset"])
            else:
                subset.append(np.nan)
            # drilldown
            if 'drillDown' in rule["config"].keys():
                drilldown.append(rule["config.drillDown"])
            else:
                drilldown.append(np.nan)
            #acceptanceMin
            if 'acceptanceMin' in rule["config"].keys():
                acceptanceMin.append(rule["config.acceptanceMin"])
            else:
                acceptanceMin.append(np.nan)

    print("archivos"            , str(len(archivos)))
    print("tablas"              , str(len(tablas)))
    print("ids"                 , str(len(ids)))
    print("namings"             , str(len(namings)))
    print("criticals"           , str(len(criticals)))
    print("types_rule"          , str(len(types_rule)))
    print("datavaluescolumn"    , str(len(datavaluescolumn)))
    print("datavaluessubset"    , str(len(datavaluessubset)))
    print("datavaluespath"      , str(len(datavaluespath)))
    print("values"              , str(len(values)))
    print("subset"              , str(len(subset)))
    print("drilldown"           , str(len(drilldown)))
    print("acceptanceMin"       , str(len(acceptanceMin)))
    df = pd.DataFrame(
    {'archivos': archivos,
     'tablas': tablas,
     'ids': ids,
     'namings': namings,
     'criticals': criticals,
     'type_rules': types_rule,
     'datavaluescolumn': datavaluescolumn,
     'datavaluessubset': datavaluessubset,
     'datavaluespath': datavaluespath,
     'values' : values,
     'subset': subset,
     'drilldown': drilldown,
     'acceptanceMin': acceptanceMin
     })
    df.to_csv("conf_info.csv", index = False)
    return df

In [3]:
dtypes = {
    'ID RULE':str
}
wip = pd.read_csv('wip_final.csv', header=1, dtype=dtypes)
wip['rule'] = wip["Type of  rule"].str.split('-').str[0]
print(wip.columns)
wip.tail(2)

Index(['IMPLANTATION TYPE', 'DEFINED BY', 'ENABLER PROJECT',
       'DOMINIO / DOMAIN', 'SUBDOMINIO / SUBDOMAIN', 'OWNERSHIP', 'DATA OWNER',
       'PROJECT WHO DEFINES', 'PROJECT ID RULE', 'ID F&T',
       'STATUS DEVELOPMENT', 'STATUS IMPLANTATION', 'REASONS NOT IMPLANTATION',
       'DEFINITION PERIOD', 'PUBLICATION DATE', 'Type of  rule',
       'Implantation Level (Field/Object)', 'Technical / Domains', 'Principle',
       'EXPERT RULE Definition Spanish', ' EXPERT RULE Definition English',
       'NAMING', 'SUPPLY TYPE *', ' C360 CODE*', 'Priorización',
       'Logical Name **', 'NOMBRE (SPA)', 'TABLE (en C360)', 'BLOQUE / LEVEL*',
       'CONCEPTUAL MODEL OBJECT 1 *', 'Secondary Table (4.3-5.2-6.4)',
       'Secondary Field (5.2-5.3-6.4)', 'Technical description of the rule',
       'Target Threshold', 'Minimum Threshold', 'Frequency of measurement',
       'FORMAT (3.2)', 'CATALOG (TAXONOMY/STATIC) (3.5-6.1-6.10)',
       'Regular Expresion', 'List of illegal values\n(3.3)',
  

,IMPLANTATION TYPE,DEFINED BY,ENABLER PROJECT,DOMINIO / DOMAIN,SUBDOMINIO / SUBDOMAIN,OWNERSHIP,DATA OWNER,PROJECT WHO DEFINES,PROJECT ID RULE,ID F&T,STATUS DEVELOPMENT,STATUS IMPLANTATION,REASONS NOT IMPLANTATION,DEFINITION PERIOD,PUBLICATION DATE,Type of rule,Implantation Level (Field/Object),Technical / Domains,Principle,EXPERT RULE Definition Spanish,EXPERT RULE Definition English,NAMING,SUPPLY TYPE *,C360 CODE*,Priorización,Logical Name **,NOMBRE (SPA),TABLE (en C360),BLOQUE / LEVEL*,CONCEPTUAL MODEL OBJECT 1 *,Secondary Table (4.3-5.2-6.4),Secondary Field (5.2-5.3-6.4),Technical description of the rule,Target Threshold,Minimum Threshold,Frequency of measurement,FORMAT (3.2),CATALOG (TAXONOMY/STATIC) (3.5-6.1-6.10),Regular Expresion,List of illegal values\n(3.3),Min Value (3.4-6.1-6.2-6.9),Max Value (3.4-6.1-6.2-6.4-6.9),Moment of Comparison (6.4-6.9),Materiality,Drilldown,Subset,Is Critical \n(True/False),With Refulsals,ID RULE,TECHNICAL STATUS,"Conf (MRField, ORField, OCField,...)",version FINISHED,Conf name,Parameters,Campo / Tabla,Campo / Tabla / Tipo de regla,Job Dataproc,UUAA,CONFIG FILE STATUS,Backlog,ID_ESTADO,BuscarV,Url conf\nv13.0.4 (new),Url Extralibs,Fecha de entrega,Origin,rule
5329,Holding,GDH,C360,NaN,NaN,NaN,NaN,NaN,NaN,NaN,FINISHED,DELIVERED,NaN,1Q25,Abr_2025,3.4-Checking the data value within the expecte...,FIELD,DOMAIN,3. Validity,El valor del campo debe ser mayor igual a 0. S...,The value of the field must be greater than 0....,gf_3p_reserves_number,NaN,NaN,NaN,NaN,NaN,t_l9ct_prod_assets_oilgas,Sustainability Information,NaN,NaN,NaN,NaN,100,100,Monthly,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,True,2891f1bb92,NEW,Url Artifactory,16.0.7,L9CT_prod-assets-oilgas_MRField.conf,CUTOFF_DATE: Date of the data,gf_3p_reserves_numbert_l9ct_prod_assets_oilgas,gf_3p_reserves_numbert_l9ct_prod_assets_oilgas...,https://bitbucket.globaldevtools.bbva.com/bitb...,L9CT,NaN,251Q,FINISHED-DELIVERED-1Q25-2891f1bb92-16.0.7,False,https://artifactory.globaldevtools.bbva.com/ui...,NaN,10/04/2025,New Flow,3.4
5330,Local,GDH,C360,NaN,NaN,NaN,NaN,NaN,NaN,NaN,DEFINED,IMPLANTABLE,NaN,2Q25,Pending,4.3-Reconciliation of the same data between di...,FIELD,DOMAIN,4. Consistency,"Comprobar que el cruce entre los campos [""gf_e...","Check that the junction between the fields [""g...",gf_entity_id,NaN,NaN,NaN,IDENTIFIER FINANCIAL ENTITY,IDENTIFICADOR ENTIDAD FINANCIERA,t_kmig_customer_allocation,Individualised Management,NaN,t_kcog_entity,gf_entity_id,NaN,100,100,Daily,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,True,289613a9ed,NEW,Url Artifactory,16.0.8,KMIG_customer-allocation_KCOG_entity _MMRule.conf,CUTOFF_DATE: Date of the data,gf_entity_idt_kmig_customer_allocation,gf_entity_idt_kmig_customer_allocation4.3-Reco...,NaN,KMIG,NEW,252Q,DEFINED-IMPLANTABLE-2Q25-289613a9ed-16.0.8,True,https://artifactory.globaldevtools.bbva.com/ui...,NaN,15/04/2025,New Flow,4.3


In [4]:
import os
#Revisamos rama de cada directorio

q = "1Q25"

ruta_confs = "C:/Users/O005820/Documents/repositorios/GDH_c360_hammurabi_UUAAs/"
ruta_confs_UA = "C:/Users/O005820/Documents/repositorios/GDH_c360_hammurabi_UUAAs/{}/glgldh360gloqlt/"
# UUAAs = ['b0hn', 'cgj7', 'cmov', 'ctar', 'dast','dtau', 'ejb6', 'g5m3', 'j7il', 'katc', 'kcdg', 'kceg', 'kcpg', 'kctd', 'kfcm', 'kmig', 'knfa', 'kpan', 'ksci', 'ksrr', 'kstb', 'kszg', 'kwth', 'l9ct', 'l9sv', 'msx9', 'nm7r', 'nols', 'p68x', 'p6r1', 'r4fu', 're4m', 'rzua', 'scat', 'tdi3', 'tlhr', 'yfvh']
UUAAs = list(wip.UUAA.str.lower().unique())
ramas = []
print('\n- Q (WIP) a analizar:\n', q)
print(UUAAs)
for UA in UUAAs:
    print(UA)
    ruta_ua = ruta_confs_UA.format(UA)
    git = 'git -C {0} rev-parse --abbrev-ref HEAD'.format(ruta_ua)
    print('- Rama Git:', os.popen(git).read())
    rama = str(os.popen(git).read()).strip()
    ramas.append(rama)


- Q (WIP) a analizar:
 1Q25
['b0hn', 'rzua', 'g5m3', 'cgj7', 'kwth', 'kszg', 'j7il', 'tdi3', 'kcpg', 'ejb6', 'yfvh', 'knfa', 'nm7r', 're4m', 'msx9', 'r4fu', 'kpan', 'katc', 'kfcm', 'p68x', 'kceg', 'l9ct', 'tlhr', 'kmig', 'nols', 'scat', 'kstb', 'ksrr', 'kcdg', 'p6r1', 'cmov', 'dast', 'ctar', 'kctd', 'dtau', 'fzpd', 'kusu', 'l9sv']
b0hn
- Rama Git: feature/MVP_2Q25_DATUM_VALIDATIONS

rzua
- Rama Git: develop

g5m3
- Rama Git: feature/MVP_1Q25_c360_10_03_2025

cgj7
- Rama Git: feature/MVP_1Q25_c360_05_03_2025

kwth
- Rama Git: develop

kszg
- Rama Git: develop

j7il
- Rama Git: master

tdi3
- Rama Git: develop

kcpg
- Rama Git: develop

ejb6
- Rama Git: develop

yfvh
- Rama Git: feature/MVP_2Q25_DATUM_VALIDATIONS

knfa
- Rama Git: feature/MVP_1Q25_c360_17_03_2025

nm7r
- Rama Git: feature/MVP_2Q25_c360_31_03_2025

re4m
- Rama Git: feature/fix_jobs

msx9
- Rama Git: feature/MVP_2Q25_DATUM_VALIDATIONS

r4fu
- Rama Git: feature/MVP_2Q25_DATUM_VALIDATIONS

kpan
- Rama Git: feature/MVP_2Q25_

In [5]:
print('Ramas unicas (siempre debemos actuar sobre una):')
print(list(np.unique(ramas)))
print('\nNº UUAAs:', len(UUAAs))

Ramas unicas (siempre debemos actuar sobre una):
[np.str_('develop'), np.str_('feature/FIX_4.3_t_l9ct_gp_cust_ext_data_comp'), np.str_('feature/MVP_1Q25_c360_05_03_2025'), np.str_('feature/MVP_1Q25_c360_10_03_2025'), np.str_('feature/MVP_1Q25_c360_17_03_2025'), np.str_('feature/MVP_1Q25_c360_18_02_2025'), np.str_('feature/MVP_1Q25_c360_27_03_2025'), np.str_('feature/MVP_2Q25_DATUM_VALIDATIONS'), np.str_('feature/MVP_2Q25_c360_31_03_2025'), np.str_('feature/fix_jobs'), np.str_('master')]

Nº UUAAs: 38


## Procesamiento

Leemos y parseamos confs

In [6]:
df_conf = get_repo_info(ruta_confs)

ConfigTree([('class', 'com.datio.hammurabi.rules.consistence.ValueConciliationRule'), ('config', ConfigTree([('id', '277e947aac'), ('column', 'g_holding_group_id'), ('keyColumns', [ConfigTree([('columnOrigin', 'g_holding_group_id'), ('columnInput', 'g_holding_group_id')])]), ('dataValues', ConfigTree([('type', 'parquet'), ('paths', ['/data/master/ksag/data/t_ksag_holding_business_group/']), ('schema', ConfigTree([('path', 'ARTIFACTORY_UNIQUE_CACHE/artifactory/SCHEMAS_REPOSITORY/schemas/COUNTRY/ksag/master/t_ksag_holding_business_group/latest/t_ksag_holding_business_group.input.schema')])), ('cache', True), ('options', ConfigTree([('overrideSchema', True), ('includeMetadataAndDeleted', True)]))])), ('dataSystemId', 'DATIO-segmentation&groups'), ('acceptanceMin', 'acceptanceMin'), ('isCritical', True), ('withRefusals', True), ('dataValuesSubset', '1==1')]))])
ConfigTree([('class', 'com.datio.hammurabi.rules.validity.NotNullValidationRule'), ('config', ConfigTree([('id', 'GL_MVP_t_b0hn_ag

In [ ]:
df_conf

,archivos,tablas,ids,namings,criticals,type_rules,datavaluescolumn,datavaluessubset,datavaluespath,values,subset,drilldown,acceptanceMin
0,C:/Users/O005820/Documents/repositorios/GDH_c3...,t_b0hn_agg_output_ems_hgl,277e947aac,g_holding_group_id,True,com.datio.hammurabi.rules.consistence.ValueCon...,NaN,1==1,[/data/master/ksag/data/t_ksag_holding_busines...,NaN,NaN,NaN,acceptanceMin
1,C:/Users/O005820/Documents/repositorios/GDH_c3...,t_b0hn_agg_output_ems_hgl,GL_MVP_t_b0hn_agg_output_ems_hgl_3.1_003,g_holding_group_id,True,com.datio.hammurabi.rules.validity.NotNullVali...,NaN,NaN,NaN,NaN,NaN,NaN,100
2,C:/Users/O005820/Documents/repositorios/GDH_c3...,t_b0hn_agg_output_ems_hgl,GL_MVP_t_b0hn_agg_output_ems_hgl_3.2_003,g_holding_group_id,True,com.datio.hammurabi.rules.validity.FormatValid...,NaN,NaN,NaN,NaN,NaN,NaN,100
3,C:/Users/O005820/Documents/repositorios/GDH_c3...,t_b0hn_agg_output_ems_hgl,GL_DOM_t_b0hn_agg_output_ems_hgl_3.1_001,gf_sc12_atrb_ems_ec_int_number,False,com.datio.hammurabi.rules.validity.NotNullVali...,NaN,NaN,NaN,NaN,NaN,NaN,100
4,C:/Users/O005820/Documents/repositorios/GDH_c3...,t_b0hn_agg_output_ems_hgl,GL_DOM_t_b0hn_agg_output_ems_hgl_3.1_002,gf_tl_atrb_ems_ec_int_number,False,com.datio.hammurabi.rules.validity.NotNullVali...,NaN,NaN,NaN,NaN,NaN,NaN,100
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5330,C:/Users/O005820/Documents/repositorios/GDH_c3...,t_yfvh_cards_calc_balances,GL_MVP_t_yfvh_cards_calc_balances_3.2_005,g_currency_id,True,com.datio.hammurabi.rules.validity.FormatValid...,NaN,NaN,NaN,NaN,NaN,NaN,100
5331,C:/Users/O005820/Documents/repositorios/GDH_c3...,t_yfvh_cards_calc_balances,GL_DOM_t_yfvh_cards_calc_balances_3.4_000,gf_cr_card_debt_capital_amount,False,com.datio.hammurabi.rules.validity.NumericRang...,NaN,NaN,NaN,NaN,NaN,NaN,100
5332,C:/Users/O005820/Documents/repositorios/GDH_c3...,t_yfvh_cards_calc_balances,GL_DOM_t_yfvh_cards_calc_balances_3.5_000,g_country_id,False,com.datio.hammurabi.rules.validity.DynamicCata...,gf_catalog_val_id,g_catalog_id ='C245' and gf_frequency_type='D',[/data/master/ktny/data/t_ktny_catalog_values_...,NaN,NaN,NaN,100
5333,C:/Users/O005820/Documents/repositorios/GDH_c3...,t_yfvh_cards_calc_balances,GL_STR_t_yfvh_cards_calc_balances_4.2_000,"[g_country_id, g_entity_id, gf_entity_id, g_co...",True,com.datio.hammurabi.rules.consistence.Duplicat...,NaN,NaN,NaN,NaN,NaN,NaN,100


Clases por regla:  
para obtener el tipo de regla a partir de la clase del conf

In [11]:
dtypes = {
    'type_rules':   str,
    'rule':         str
         }
clases = pd.read_csv('clases_regla.csv',dtype = dtypes)[['type_rules','rule']]
df_conf_rule = df_conf.merge(clases, on='type_rules',how='left')
df_conf_rule.criticals[df_conf_rule.criticals=="IS_VOL_CRITICAL"] = True #lee el parámetro opcional, todos son True en realidad
df_conf_rule.to_csv('comprobaciones/extraccion_reglas.csv',index=False)
df_conf_rule.tail(2)

C:\Users\O005820\AppData\Local\Temp\ipykernel_12984\3193084538.py:7: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df_conf_rule.criticals[df_conf_rule.criticals=="IS_VOL_CRITICAL"] = True #lee el parámetro opcional, todos son True en realida

,archivos,tablas,ids,namings,criticals,type_rules,datavaluescolumn,datavaluessubset,datavaluespath,values,subset,drilldown,acceptanceMin,rule
5333,C:/Users/O005820/Documents/repositorios/GDH_c3...,t_yfvh_cards_calc_balances,GL_STR_t_yfvh_cards_calc_balances_4.2_000,"[g_country_id, g_entity_id, gf_entity_id, g_co...",True,com.datio.hammurabi.rules.consistence.Duplicat...,NaN,NaN,NaN,NaN,NaN,NaN,100,4.2
5334,C:/Users/O005820/Documents/repositorios/GDH_c3...,t_yfvh_cards_calc_balances,GL_STR_t_yfvh_cards_calc_balances_6.9_000,NaN,True,com.datio.hammurabi.rules.accuracy.VolumetricT...,NaN,gf_cutoff_date='FECHA_VOLUMETRIC',[/data/master/yfvh/data/t_yfvh_cards_calc_bala...,NaN,NaN,NaN,NaN,6.9


Leemos última version del WIP

## Análisis

In [7]:
print('¿Coincide el número de reglas?\n',len(df_conf_rule) == len(wip))
print('\n- wip:  ',len(wip))
print('- repos:',len(df_conf_rule))

¿Coincide el número de reglas?
 True

- wip:   5146
- repos: 5146


**Coinciden todos los ids en el wip - conf?**  
*id*  
guardado en `./comprobaciones/id/`

In [8]:
df_union_id = df_conf[['ids']].merge(wip[['ID RULE']], left_on="ids", right_on='ID RULE', how='outer', indicator=True)
df_union_id.query('_merge=="left_only"').to_csv('./comprobaciones/id/id-CONF_ONLY.csv',sep=';',index=False)
df_union_id.query('_merge=="right_only"').to_csv('./comprobaciones/id/id-WIP_ONLY.csv',sep=';',index=False)
print('¿Están todos los Ids representados en el WIP y al contrario?\n',df_union_id._merge.nunique()==1)
print('\n-Numeros:')
print(df_union_id._merge.value_counts())
df_union_id.query('_merge=="right_only"')

¿Están todos los Ids representados en el WIP y al contrario?
 True

-Numeros:
both          5146
left_only        0
right_only       0
Name: _merge, dtype: int64


,ids,ID RULE,_merge


In [9]:
# df_union_id.query('_merge=="left_only"')

**CONTEO: Nº Reglas técnicas / DOM**

In [10]:
print('¿CORRECTO?\n',df_conf_rule.criticals.value_counts()[0] == wip['Is Critical \n(True/False)'].value_counts()[0])
print('\n- CONFs')
print(df_conf_rule.criticals.value_counts())
print('\n- WIP')
print(wip['Is Critical \n(True/False)'].value_counts())

¿CORRECTO?
 True

- CONFs
True     3328
False    1818
Name: criticals, dtype: int64

- WIP
True     3328
False    1818
Name: Is Critical \n(True/False), dtype: int64


**id, tabla, tipo, iscritical**  
*id_tabla_tipo_critical*  
guardado en `./comprobaciones/id_tabla_tipo_critical/`

In [11]:
wip['id_tabla_tipo_critical']           = wip['ID RULE'] +'-'+ wip['TABLE (en C360)'].str.strip() +'-'+ wip['rule']  +'-'+ wip['Is Critical \n(True/False)'].astype(str)
wip['naming_clean']                     = wip['NAMING'].str.replace("[ \n\,\s+]",',',regex=True).str.replace("\,+",',',regex=True).str.strip(',').replace('Rule,at,Object,level','').replace('nan','').fillna('')
wip['naming_set']                       = wip['naming_clean'].str.split(',').apply(lambda x:set(x))
wip['naming_hash']                      = wip['naming_set'].apply(lambda x:hash(frozenset(x)))

df_conf_rule['id_tabla_tipo_critical']  = df_conf_rule['ids'] +'-'+ df_conf_rule['tablas'].str.strip() +'-'+ df_conf_rule['rule']  +'-'+ df_conf_rule['criticals'].astype(str)
df_conf_rule['naming_clean']            = df_conf_rule['namings'].astype(str).str.replace("[\[\]\s\']",',',regex=True).str.replace("\,+",',',regex=True).replace('nan','').str.strip(',').fillna('')
df_conf_rule['naming_set']              = df_conf_rule['naming_clean'].str.split(',').apply(lambda x:set(x))
df_conf_rule['naming_hash']             = df_conf_rule['naming_set'].apply(lambda x:hash(frozenset(x)))


col = ['id_tabla_tipo_critical']
df_union_id_tabla_tipo_critical = df_conf_rule[col].merge(wip[col], on=col, how='outer', indicator=True)
df_union_id_tabla_tipo_critical = pd.concat([df_union_id_tabla_tipo_critical, df_union_id_tabla_tipo_critical.id_tabla_tipo_critical.str.split('-',expand=True)], axis=1)
df_union_id_tabla_tipo_critical.query('_merge=="left_only"').to_csv('./comprobaciones/id_tabla_tipo_critical/id_tabla_tipo_critical-CONF_ONLY.csv',sep=';',index=False)
df_union_id_tabla_tipo_critical.query('_merge=="right_only"').to_csv('./comprobaciones/id_tabla_tipo_critical/id_tabla_tipo_critical-WIP_ONLY.csv',sep=';',index=False)
print('¿CORRECTO?\n',df_union_id_tabla_tipo_critical._merge.nunique()==1)
print('\n- Numeros:')
print(df_union_id_tabla_tipo_critical._merge.value_counts())
df_union_id_tabla_tipo_critical.query('_merge=="right_only"')

¿CORRECTO?
 True

- Numeros:
both          5146
left_only        0
right_only       0
Name: _merge, dtype: int64


,id_tabla_tipo_critical,_merge,0,1,2,3


In [12]:
df_union_id_tabla_tipo_critical.query('_merge=="left_only"')

,id_tabla_tipo_critical,_merge,0,1,2,3


**id, tabla, tipo, iscritical, naming**  
*id_tabla_tipo_critical_naming*  
guardado en `./comprobaciones/id_tabla_tipo_critical_naming/`

> **ATENCIÓN!**: las **reglas 5.6** funcionan diferente y no coincidirán en los naming, ya que la columna principal va en `referenceColumn`  
> (por ahora tenemos censados 2)

In [13]:
cols = ['id_tabla_tipo_critical','naming_set','naming_hash']
cols_on = ['id_tabla_tipo_critical','naming_hash']
df_union_id_tabla_tipo_critical_naming = df_conf_rule[cols].merge(wip[cols], on=cols_on, how='outer', indicator=True)
df_union_id_tabla_tipo_critical_naming = pd.concat([df_union_id_tabla_tipo_critical_naming, df_union_id_tabla_tipo_critical_naming.id_tabla_tipo_critical.str.split('-',expand=True)], axis=1)
df_union_id_tabla_tipo_critical_naming.query('_merge=="left_only"').to_csv('./comprobaciones/id_tabla_tipo_critical_naming/id_tabla_tipo_critical_naming-CONF_ONLY.csv',sep=';',index=False)
df_union_id_tabla_tipo_critical_naming.query('_merge=="right_only"').to_csv('./comprobaciones/id_tabla_tipo_critical_naming/id_tabla_tipo_critical_naming-WIP_ONLY.csv',sep=';',index=False)
print('¿CORRECTO?\n',df_union_id_tabla_tipo_critical_naming._merge.nunique()==1)
print('\n- Numeros:')
print(df_union_id_tabla_tipo_critical_naming._merge.value_counts())
df_union_id_tabla_tipo_critical_naming.query('_merge=="left_only"')

¿CORRECTO?
 False

- Numeros:
both          5144
left_only        2
right_only       2
Name: _merge, dtype: int64


,id_tabla_tipo_critical,naming_set_x,naming_hash,naming_set_y,_merge,0,1,2,3
4686,GL_DOM_t_rzua_contract_output_emissions_5.6_00...,"{g_country_id, gf_entity_id}",7675798404867042670,NaN,left_only,GL_DOM_t_rzua_contract_output_emissions_5.6_000,t_rzua_contract_output_emissions,5.6,False
4953,GL_DOM_t_rzua_sust_operations_5.6_000-t_rzua_s...,"{g_country_id, gf_entity_id}",7675798404867042670,NaN,left_only,GL_DOM_t_rzua_sust_operations_5.6_000,t_rzua_sust_operations,5.6,False


**catalogos y frecuencia 3.5**  
*3_5*  
guardado en `./comprobaciones/3_5/`

In [14]:
cols_3_5_wip = ['ID RULE','CATALOG (TAXONOMY/STATIC) (3.5-6.1-6.10)','Frequency of measurement','Subset','DEFINITION PERIOD','PUBLICATION DATE','STATUS DEVELOPMENT','STATUS IMPLANTATION','Parameters']
wip_3_5 = wip.query('rule == "3.5"')[cols_3_5_wip]

cols_3_5_confs = ['ids','naming_set','tablas','subset','datavaluespath','values','datavaluescolumn','datavaluessubset']
df_conf_rule_3_5_din = df_conf_rule.query('(rule == "3.5") & (type_rules=="com.datio.hammurabi.rules.validity.DynamicCatalogRule")')[cols_3_5_confs].merge(wip_3_5, left_on='ids', right_on= 'ID RULE', how='left')
df_conf_rule_3_5_din['datavaluespath_str'] = df_conf_rule_3_5_din['datavaluespath'].apply(lambda x: x[0])

df_conf_rule_3_5_estatic = df_conf_rule.query('(rule == "3.5") & (type_rules=="com.datio.hammurabi.rules.validity.StaticCatalogRule")')[cols_3_5_confs].merge(wip_3_5, left_on='ids', right_on= 'ID RULE', how='left')
df_conf_rule_3_5_ktny_catalog_values = df_conf_rule_3_5_din.query('datavaluespath_str.str.contains("t_ktny_catalog_values_taxonomy")').copy()
df_conf_rule_3_5_ktny_rel_values = df_conf_rule_3_5_din.query('datavaluespath_str.str.contains("t_ktny_rel_values_taxonomy")').copy()
df_conf_rule_3_5_noktny = df_conf_rule_3_5_din.query('~datavaluespath_str.str.contains("t_ktny")').copy()

#extraemos campos de catalogo
df_conf_rule_3_5_ktny_catalog_values['datavaluessubset_gcatalogid'] = df_conf_rule_3_5_ktny_catalog_values.datavaluessubset.str.extract(r"g_catalog_id ?= ?\'(.{0,10})\'")
df_conf_rule_3_5_ktny_catalog_values['datavaluessubset_gffrequencytype'] = df_conf_rule_3_5_ktny_catalog_values.datavaluessubset.str.extract(r"gf_frequency_type ?= ?\'(.{0,10})\'")

df_conf_rule_3_5_ktny_rel_values['datavaluessubset_gfinitialcatalogid'] = df_conf_rule_3_5_ktny_rel_values.datavaluessubset.str.extract(r"gf_initial_catalog_id ?= ?\'(.{0,10})\' ")
df_conf_rule_3_5_ktny_rel_values['datavaluessubset_gffinalcatalogid '] = df_conf_rule_3_5_ktny_rel_values.datavaluessubset.str.extract(r"gf_final_catalog_id  ?= ?\'(.{0,10})\' ")
df_conf_rule_3_5_ktny_rel_values['datavaluessubset_gfinitialcatalogvalid '] = df_conf_rule_3_5_ktny_rel_values.datavaluessubset.str.extract(r"gf_initial_catalog_val_id  ?= ?\'(.{0,10})\' ")
df_conf_rule_3_5_ktny_rel_values['datavaluessubset_gffrequencytype'] = df_conf_rule_3_5_ktny_rel_values.datavaluessubset.str.extract(r"gf_frequency_type ?= ?\'(.{0,10})\'")

#guardamos
df_conf_rule_3_5_estatic.to_csv('comprobaciones/3_5/conf_rule_3_5_estatic.csv',sep = ";", index=False)
df_conf_rule_3_5_ktny_catalog_values.to_csv('comprobaciones/3_5/conf_rule_3_5_ktny_catalog_values.csv',sep = ";", index=False)
df_conf_rule_3_5_ktny_rel_values.to_csv('comprobaciones/3_5/conf_rule_3_5_ktny_rel_values.csv',sep = ";", index=False)
df_conf_rule_3_5_noktny.to_csv('comprobaciones/3_5/conf_rule_3_5_noktny.csv',sep = ";", index=False)

print('= Total WIP - Repo =')
print('====================')
print('Total reglas 3.5 catálogo estático:',len(df_conf_rule_3_5_estatic))
print('Total reglas 3.5 catálogo dinámico:',len(df_conf_rule_3_5_din))
print('- KTNY catalog values:',len(df_conf_rule_3_5_ktny_catalog_values))
print('- KTNY rel values:',len(df_conf_rule_3_5_ktny_rel_values))
print('- otros NO KTNY:',len(df_conf_rule_3_5_noktny))
print('\notros catalogos:\n',df_conf_rule_3_5_noktny.datavaluespath_str.unique())

# filtramos Q actual
df_conf_rule_3_5_estatic[df_conf_rule_3_5_estatic['DEFINITION PERIOD'] == q].to_csv('comprobaciones/3_5/Q_actual/conf_rule_3_5_estatic.csv',sep = ";", index=False)
df_conf_rule_3_5_ktny_catalog_values[df_conf_rule_3_5_ktny_catalog_values['DEFINITION PERIOD'] == q].to_csv('comprobaciones/3_5/Q_actual/conf_rule_3_5_ktny_catalog_values.csv',sep = ";", index=False)
df_conf_rule_3_5_ktny_rel_values[df_conf_rule_3_5_ktny_rel_values['DEFINITION PERIOD'] == q].to_csv('comprobaciones/3_5/Q_actual/conf_rule_3_5_ktny_rel_values.csv',sep = ";", index=False)
df_conf_rule_3_5_noktny[df_conf_rule_3_5_noktny['DEFINITION PERIOD'] == q].to_csv('comprobaciones/3_5/Q_actual/conf_rule_3_5_noktny.csv',sep = ";", index=False)


print('\n= Q ACTUAL =')
print('============')
print('Total reglas 3.5 catálogo estático:',len(df_conf_rule_3_5_estatic[df_conf_rule_3_5_estatic['DEFINITION PERIOD'] == q]))
print('Total reglas 3.5 catálogo dinámico:',len(df_conf_rule_3_5_din[df_conf_rule_3_5_din['DEFINITION PERIOD'] == q]))
print('- KTNY catalog values:',len(df_conf_rule_3_5_ktny_catalog_values[df_conf_rule_3_5_ktny_catalog_values['DEFINITION PERIOD'] == q]))
print('- KTNY rel values:',len(df_conf_rule_3_5_ktny_rel_values[df_conf_rule_3_5_ktny_rel_values['DEFINITION PERIOD'] == q]))
print('- otros NO KTNY:',len(df_conf_rule_3_5_noktny[df_conf_rule_3_5_noktny['DEFINITION PERIOD'] == q]))
print('\notros catalogos:\n',df_conf_rule_3_5_noktny[df_conf_rule_3_5_noktny['DEFINITION PERIOD'] == q].datavaluespath_str.unique())

= Total WIP - Repo =
Total reglas 3.5 catálogo estático: 138
Total reglas 3.5 catálogo dinámico: 899
- KTNY catalog values: 738
- KTNY rel values: 19
- otros NO KTNY: 142

otros catalogos:
 ['MASTERSCHEMA.CATALOG_TABLE_0' 'MASTERSCHEMA.CATALOG_TABLE_1'
 'MASTERSCHEMA.CATALOG_TABLE_2' 'MASTERSCHEMA.CATALOG_TABLE_3'
 'MASTERSCHEMA.CATALOG_TABLE_4' 'MASTERSCHEMA.CATALOG_TABLE_5'
 'MASTERSCHEMA.CATALOG_TABLE_6' 'MASTERSCHEMA.CATALOG_TABLE_7'
 'MASTERSCHEMA.CATALOG_TABLE_8' 'MASTERSCHEMA.CATALOG_TABLE_9'
 'MASTERSCHEMA.CATALOG_TABLE_10' 'MASTERSCHEMA.CATALOG_TABLE_11'
 '/data/master/p68x/data/t_p68x_insights_relations']

= Q ACTUAL =
Total reglas 3.5 catálogo estático: 6
Total reglas 3.5 catálogo dinámico: 72
- KTNY catalog values: 69
- KTNY rel values: 0
- otros NO KTNY: 3

otros catalogos:
 ['MASTERSCHEMA.CATALOG_TABLE_0' 'MASTERSCHEMA.CATALOG_TABLE_7']


In [15]:
df_conf_rule_3_5_estatic_Q = df_conf_rule_3_5_estatic[df_conf_rule_3_5_estatic['DEFINITION PERIOD'] == q][['ids','values','CATALOG (TAXONOMY/STATIC) (3.5-6.1-6.10)']]
df_conf_rule_3_5_estatic_Q['valuesWIP'] = df_conf_rule_3_5_estatic_Q['CATALOG (TAXONOMY/STATIC) (3.5-6.1-6.10)'].str.split('/')
df_conf_rule_3_5_estatic_Q['coincide'] = df_conf_rule_3_5_estatic_Q['valuesWIP'].apply(lambda x: set(x)) == df_conf_rule_3_5_estatic_Q['values'].apply(lambda x: set(x))
print('¿Coinciden parámetros estáticos?')
df_conf_rule_3_5_estatic_Q.coincide.value_counts()
df_conf_rule_3_5_estatic_Q.query('coincide == False')

¿Coinciden parámetros estáticos?


,ids,values,CATALOG (TAXONOMY/STATIC) (3.5-6.1-6.10),valuesWIP,coincide


In [16]:
df_conf_rule_3_5_ktny_catalog_values_Q = df_conf_rule_3_5_ktny_catalog_values[df_conf_rule_3_5_ktny_catalog_values['DEFINITION PERIOD'] == q][['ids','tablas','datavaluessubset','CATALOG (TAXONOMY/STATIC) (3.5-6.1-6.10)','Frequency of measurement','datavaluessubset_gcatalogid','datavaluessubset_gffrequencytype']]
df_conf_rule_3_5_ktny_catalog_values_Q['Frequency of measurement'] = df_conf_rule_3_5_ktny_catalog_values_Q['Frequency of measurement'].str[0]
df_conf_rule_3_5_ktny_catalog_values_Q['coincide_freq'] = df_conf_rule_3_5_ktny_catalog_values_Q['Frequency of measurement'].str.strip()  == df_conf_rule_3_5_ktny_catalog_values_Q['datavaluessubset_gffrequencytype'].str.strip()
df_conf_rule_3_5_ktny_catalog_values_Q['coincide_cat'] = df_conf_rule_3_5_ktny_catalog_values_Q['CATALOG (TAXONOMY/STATIC) (3.5-6.1-6.10)'].str.strip() == df_conf_rule_3_5_ktny_catalog_values_Q['datavaluessubset_gcatalogid'].str.strip()
df_conf_rule_3_5_ktny_catalog_values_Q['coincide'] = df_conf_rule_3_5_ktny_catalog_values_Q['coincide_freq'] * df_conf_rule_3_5_ktny_catalog_values_Q['coincide_cat']
print('¿Coinciden parámetros dinámicos?')
print(df_conf_rule_3_5_ktny_catalog_values_Q.coincide.value_counts())
print('Frecuencia?')
print(df_conf_rule_3_5_ktny_catalog_values_Q.coincide_freq.value_counts())
print('Catalogo')
print(df_conf_rule_3_5_ktny_catalog_values_Q.coincide_cat.value_counts())


¿Coinciden parámetros dinámicos?
True     66
False     3
Name: coincide, dtype: int64
Frecuencia?
True     66
False     3
Name: coincide_freq, dtype: int64
Catalogo
True    69
Name: coincide_cat, dtype: int64


In [17]:
df_conf_rule_3_5_ktny_catalog_values_Q.query('coincide_cat == False')

,ids,tablas,datavaluessubset,CATALOG (TAXONOMY/STATIC) (3.5-6.1-6.10),Frequency of measurement,datavaluessubset_gcatalogid,datavaluessubset_gffrequencytype,coincide_freq,coincide_cat,coincide


In [18]:
df_conf_rule_3_5_ktny_catalog_values_Q.query('coincide_freq == False')

,ids,tablas,datavaluessubset,CATALOG (TAXONOMY/STATIC) (3.5-6.1-6.10),Frequency of measurement,datavaluessubset_gcatalogid,datavaluessubset_gffrequencytype,coincide_freq,coincide_cat,coincide
642,27927468f3,t_l9ct_em_factors_buildings,g_catalog_id ='B347' and gf_frequency_type='D',B347,P,B347,D,False,True,False
644,2792746377,t_l9ct_em_factors_by_sector,g_catalog_id ='B347' and gf_frequency_type='D',B347,P,B347,D,False,True,False
649,2792746658,t_l9ct_em_factors_motor_ve,g_catalog_id ='B347' and gf_frequency_type='D',B347,P,B347,D,False,True,False


In [19]:
df_conf_rule_3_5_ktny_catalog_values_Q.query('coincide == False')

,ids,tablas,datavaluessubset,CATALOG (TAXONOMY/STATIC) (3.5-6.1-6.10),Frequency of measurement,datavaluessubset_gcatalogid,datavaluessubset_gffrequencytype,coincide_freq,coincide_cat,coincide
642,27927468f3,t_l9ct_em_factors_buildings,g_catalog_id ='B347' and gf_frequency_type='D',B347,P,B347,D,False,True,False
644,2792746377,t_l9ct_em_factors_by_sector,g_catalog_id ='B347' and gf_frequency_type='D',B347,P,B347,D,False,True,False
649,2792746658,t_l9ct_em_factors_motor_ve,g_catalog_id ='B347' and gf_frequency_type='D',B347,P,B347,D,False,True,False


In [20]:
df_conf_rule_3_5_noktny_Q = df_conf_rule_3_5_noktny[df_conf_rule_3_5_noktny['DEFINITION PERIOD'] == q][['ids','tablas','datavaluespath','CATALOG (TAXONOMY/STATIC) (3.5-6.1-6.10)','Parameters']]
print('¿Está bien informado en el WIP?')
print(df_conf_rule_3_5_noktny_Q['CATALOG (TAXONOMY/STATIC) (3.5-6.1-6.10)'].value_counts())
df_conf_rule_3_5_noktny_Q['param_tables'] = df_conf_rule_3_5_noktny_Q.Parameters.str.extract(r'(CATALOG_TABLE_\d{1,2})')
df_conf_rule_3_5_noktny_Q['tables'] = df_conf_rule_3_5_noktny_Q.datavaluespath.apply(lambda x: x[0]).str.split('.').apply(lambda x: x[1])
df_conf_rule_3_5_noktny_Q['coincide'] = df_conf_rule_3_5_noktny_Q['param_tables'] == df_conf_rule_3_5_noktny_Q['tables']
print()
print('¿Coinciden parámetros informados?')
print(df_conf_rule_3_5_noktny_Q['coincide'].value_counts())
df_conf_rule_3_5_noktny_Q.query('coincide == False')

¿Está bien informado en el WIP?
LOCAL: La definición del catálogo a usar dependerá del equipo de Gobierno local    3
Name: CATALOG (TAXONOMY/STATIC) (3.5-6.1-6.10), dtype: int64

¿Coinciden parámetros informados?
True    3
Name: coincide, dtype: int64


,ids,tablas,datavaluespath,CATALOG (TAXONOMY/STATIC) (3.5-6.1-6.10),Parameters,param_tables,tables,coincide


***subset***  
guardado en `./comprobaciones/subset/`

In [21]:
df_subset = df_conf_rule[['ids','subset','naming_set','tablas']].merge(wip[['ID RULE','Subset','DEFINITION PERIOD']], left_on='ids',right_on='ID RULE', how='left').dropna(subset=['subset','Subset'],how='all')

df_subset.to_csv('comprobaciones/subset/subset.csv',sep = ";", index=False)
df_subset[df_subset['DEFINITION PERIOD'] == q].to_csv('comprobaciones/subset/Q_actual/subset.csv',sep = ";", index=False)

print('\n= Total WIP - Repo =')
print('============')
print('Total reglas con subset:',len(df_subset))

print('\n= Q ACTUAL =')
print('============')
print('Total reglas con subset:',len(df_subset[df_subset['DEFINITION PERIOD'] == q]))


= Total WIP - Repo =
Total reglas con subset: 821

= Q ACTUAL =
Total reglas con subset: 49


In [22]:
df_subset_Q = df_subset[df_subset['DEFINITION PERIOD'] == q][['ids','tablas','subset','Subset']]
df_subset_Q[df_subset_Q.subset.isnull()]

,ids,tablas,subset,Subset


In [23]:
df_subset_Q[['subset','Subset']].set_index('subset').to_dict()['Subset']

{'gf_vehicle_horse_power_ex1_number is not null': 'Distinto de Null',
 'gf_company_ems_cyr_number is not null': 'Si el campo (gf_company_ems_cyr_number) es diferente a NULL',
 'gf_aligned_ems_cyr_number is not null': 'Si el campo (gf_aligned_ems_cyr_number) es diferente a NULL',
 'gf_ems_align_deg_cyr_number is not null': 'Si el campo (gf_ems_align_deg_cyr_number) es diferente a NULL',
 "g_holding_group_id is not null and g_holding_group_id != ''": 'La regla solo debe ejecutarse cuando el campo g_holding_group_id esté rellenado. ',
 'g_green_bond_ind_type is not null': 'No debe aplicarse cuando el campo sea nulo.',
 'g_eu_taxo_green_bond_ind_type is not null': 'No debe aplicarse cuando el campo sea nulo.',
 'g_icma_stds_green_bnd_ind_type is not null': 'No debe aplicarse cuando el campo sea nulo.',
 'g_esg_bond_type_id is not null': 'No debe aplicarse cuando el campo sea nulo.',
 "gf_main_contact_type is not null and gf_main_contact_type != ''": 'Distinto de Null o vacío',
 "g_sust_sta

**drillDown**  
guardado en `./comprobaciones/drilldown/`

In [24]:
# pendiente 1Q24
df_drilldown = df_conf_rule[['ids','drilldown','tablas']].merge(wip[['ID RULE','Drilldown','DEFINITION PERIOD']], left_on='ids',right_on='ID RULE', how='left').dropna(subset=['drilldown','Drilldown'], how='all')

df_drilldown.to_csv('comprobaciones/drilldown/drilldown.csv',sep = ";", index=False)
df_drilldown[df_drilldown['DEFINITION PERIOD'] == q].to_csv('comprobaciones/drilldown/Q_actual/drilldown.csv',sep = ";", index=False)

print('\n= Total WIP - Repo =')
print('============')
print('Total reglas con drillDown:',len(df_drilldown))

print('\n= Q ACTUAL =')
print('============')
print('Total reglas con drillDown:',len(df_drilldown[df_drilldown['DEFINITION PERIOD'] == q]))


= Total WIP - Repo =
Total reglas con drillDown: 40

= Q ACTUAL =
Total reglas con drillDown: 0


In [25]:
df_drilldown_Q = df_drilldown[df_drilldown['DEFINITION PERIOD'] == q][['ids','tablas','drilldown','Drilldown']]
df_drilldown_Q['Drilldown'] = df_drilldown_Q.Drilldown.str.replace('\s','',regex=True).str.split(',')
df_drilldown_Q['coincide'] = df_drilldown_Q.Drilldown.apply(lambda x: set(x)) == df_drilldown_Q.drilldown.apply(lambda x: set(x))
df_drilldown_Q.query('coincide == False')

,ids,tablas,drilldown,Drilldown,coincide


UUAAs a mergear

In [26]:
wip[(wip['STATUS DEVELOPMENT']=="FINISHED") & (wip['STATUS IMPLANTATION']=="IMPLANTABLE")].UUAA.sort_values().unique().tolist()

[]